In [62]:
from IPython.core.interactiveshell import InteractiveShell
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
import json 
from sklearn.feature_extraction.text import TfidfTransformer
from pandas.io.json import json_normalize

In [63]:
dt=pd.read_json('Digital_Music_5.json',lines=True)
dt.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,5555991584,"[3, 3]",5,"It's hard to believe ""Memory of Trees"" came ou...","09 12, 2006",A3EBHHCZO6V2A4,"Amaranth ""music fan""",Enya's last great album,1158019200
1,5555991584,"[0, 0]",5,"A clasically-styled and introverted album, Mem...","06 3, 2001",AZPWAXJG9OJXV,bethtexas,Enya at her most elegant,991526400
2,5555991584,"[2, 2]",5,I never thought Enya would reach the sublime h...,"07 14, 2003",A38IRL0X2T4DPF,bob turnley,The best so far,1058140800
3,5555991584,"[1, 1]",5,This is the third review of an irish album I w...,"05 3, 2000",A22IK3I6U76GX0,Calle,Ireland produces good music.,957312000
4,5555991584,"[1, 1]",4,"Enya, despite being a successful recording art...","01 17, 2008",A1AISPOIIHTHXX,"Cloud ""...""",4.5; music to dream to,1200528000


In [64]:
dt["Rating"]=1

Chia đánh giá theo thang điểm của overall :  
1,2: negative
3  : neural
4,5: positive

In [65]:
dt["Rating"][(dt['overall'] > 0) & (dt['overall']<3)]= "negative"
dt["Rating"][(dt['overall'] == 3)]= "neural"
dt["Rating"][(dt['overall']>3)&(dt['overall']<6)]="positive"

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
dt.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,Rating
0,5555991584,"[3, 3]",5,"It's hard to believe ""Memory of Trees"" came ou...","09 12, 2006",A3EBHHCZO6V2A4,"Amaranth ""music fan""",Enya's last great album,1158019200,positive
1,5555991584,"[0, 0]",5,"A clasically-styled and introverted album, Mem...","06 3, 2001",AZPWAXJG9OJXV,bethtexas,Enya at her most elegant,991526400,positive
2,5555991584,"[2, 2]",5,I never thought Enya would reach the sublime h...,"07 14, 2003",A38IRL0X2T4DPF,bob turnley,The best so far,1058140800,positive
3,5555991584,"[1, 1]",5,This is the third review of an irish album I w...,"05 3, 2000",A22IK3I6U76GX0,Calle,Ireland produces good music.,957312000,positive
4,5555991584,"[1, 1]",4,"Enya, despite being a successful recording art...","01 17, 2008",A1AISPOIIHTHXX,"Cloud ""...""",4.5; music to dream to,1200528000,positive


In [67]:
dt.dtypes

asin              object
helpful           object
overall            int64
reviewText        object
reviewTime        object
reviewerID        object
reviewerName      object
summary           object
unixReviewTime     int64
Rating            object
dtype: object

In [68]:
dt.describe().T

,count,mean,std,min,25%,50%,75%,max
overall,64706.0,4.222514e+00,1.086081e+00,1.0,4.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00
unixReviewTime,64706.0,1.151117e+09,1.279250e+08,893721600.0,1.056931e+09,1.128989e+09,1.245024e+09,1.406074e+09


In [91]:
dt['Rating'].value_counts()

positive    52116
neural       6789
negative     5801
Name: Rating, dtype: int64

Dữ liệu cho thấy chênh lệch giữa positive,neural,negative là khá lớn tỉ lệ lần lượt là 0.8:0.1:0.1

Vì mình chỉ làm trên 2 columns là reviewText và overall nên chỉ kiểm tra có dòng dl nào null trên 2 columns này thôi

In [70]:
dt[dt.reviewText.isnull()]

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,Rating


In [71]:
dt[dt.overall.isnull()]

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,Rating


Tạo tập Train từ 1/2 dữ liệu review và Rating

In [72]:
vectorizer=CountVectorizer()
train_data=dt['reviewText'].head(32000).tolist()

Đếm words để thống kê tầng suất của words trong cột reviewText

In [73]:
train_counts=vectorizer.fit_transform(train_data)

Dữ liệu review train được đưa qua tfidf để loại các từ có tầng suất cao như :a,the,it,and.....

In [74]:
tfidf_transformer = TfidfTransformer()
x_train = tfidf_transformer.fit_transform(train_counts)
y_train = dt['Rating'].head(32000)

In [75]:
clf = MultinomialNB().fit(x_train, y_train)

Sau khi tạo model predict từ tập train ,tiếp theo tạo test data từ 1 nửa còn lại của dữ liệu

In [76]:
data_test=dt['reviewText'].tail(32000).tolist()
y_test = dt['Rating'].tail(32000)

Sử dụng tfidf tương tự cho tập test và chạy model trên tập test và so sánh kết quả

In [77]:
test_counts = vectorizer.transform(data_test)
x_test = tfidf_transformer.transform(test_counts)
predicted = clf.predict(x_test)
print('%.1f%%' % (np.mean(predicted == y_test) * 100))

74.3%


Tỉ lệ cho thấy model không quá overfiting, nhưng vì sự chênh lệch của số lượng của positive,neural,negative nên có thể model không chính xác.

Tiếp theo ,Mình làm model predict mà không có TFIDF và so sánh kết quả

In [89]:
vectorizer=CountVectorizer()
train_data=dt['reviewText'].head(32000).tolist()
x_train1=vectorizer.fit_transform(train_data)
y_train1 = dt['Rating'].head(32000)
clf = MultinomialNB().fit(x_train1, y_train1)

In [90]:
data_test=dt['reviewText'].tail(32000).tolist()
y_test1 = dt['Rating'].tail(32000)
x_test1 = vectorizer.transform(data_test)
predicted = clf.predict(x_test1)
print('%.1f%%' % (np.mean(predicted == y_test1) * 100))

75.2%


In [ ]:
So với có tfidf thì model này có kết quả gần bằng ,không khác biệt nhiều.

Kết luận:
Sử dụng model Naive Bayes để classification bình luận trong tập Digital_Music cho ra kết quả tương đối , vì tỉ lệ category khác nhau rất lớn nên có thể tự động predict là positive thì vẫn đúng đến 80% trường hợp , vì vậy cần 1 tập dữ liệu lớn hơn 